<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

<h1>---------------------------------------------------Resnet50 Model --------------------------------------------------

### These Steps can be skipped and by using load model function we can load the previously trained model and then can evaluate it on dataset of week4

<h2>Download Data

In [1]:
## get the data
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

--2020-07-10 21:44:02--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261482368 (249M) [application/zip]
Saving to: ‘concrete_data_week3.zip.3’

concrete_data_week3 100%[===================>] 249.37M  30.2MB/s    in 8.2s    

2020-07-10 21:44:11 (30.4 MB/s) - ‘concrete_data_week3.zip.3’ saved [261482368/261482368]



In [ ]:
!unzip concrete_data_week3.zip

<h2>Import libraries

In [3]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
print(keras.__version__)

Using TensorFlow backend.


2.3.1


<h2>Define Global Variable

Here, we will define constants that we will be using throughout the rest of the lab. 

1. We are obviously dealing with two classes, so *num_classes* is 2. 
2. The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3. We will training and validating the model using batches of 100 images.

In [4]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

## Construct ImageDataGenerator Instances

In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to *preprocess_input* which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.

In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Next, we will use the *flow_from_directory* method to get the training images as follows:

In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


**Your Turn**: Use the *flow_from_directory* method to get the validation images and assign the result to **validation_generator**.

In [7]:
## Type your answer here
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 10001 images belonging to 2 classes.


## Build, Compile and Fit Model

In this section, we will start building our model. We will use the Sequential model class from Keras.

In [8]:
model = Sequential()
#model = tensorflow.keras.Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument *include_top* and set it to **False**.

In [9]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Then, we will define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.

In [10]:
model.add(Dense(num_classes, activation='softmax'))

You can access the model's layers using the *layers* attribute of our model object. 

In [11]:
model.layers

You can see that our model is composed of two sets of layers. The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.

You can access the ResNet50 layers by running the following:

In [12]:
model.layers[0].layers

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.

In [13]:
model.layers[0].trainable = False

And now using the *summary* attribute of the model, we can see how many parameters we will need to optimize in order to train the output layer.

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Next we compile our model using the **adam** optimizer.

In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:

In [16]:
steps_per_epoch_training = len(train_generator)/ batch_size_training
steps_per_epoch_validation = len(validation_generator)/ batch_size_validation
num_epochs = 2

Finally, we are ready to start training our model. Unlike a conventional deep learning training were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the **fit_generator** method.

In [17]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
4/3 [=======================================] - 13s 3s/step - loss: 0.7089 - accuracy: 0.6050 - val_loss: 0.6476 - val_accuracy: 0.6850
Epoch 2/2
4/3 [=======================================] - 3s 730ms/step - loss: 0.4301 - accuracy: 0.8100 - val_loss: 0.5127 - val_accuracy: 0.7100


Now that the model is trained, you are ready to start using it to classify images.

Since training can take a long time when building deep learning models, it is always a good idea to save your model once the training is complete if you believe you will be using the model again later. You will be using this model in the next module, so go ahead and save your model.

In [18]:
model.save('classifier_resnet_model.h5')

<h1>----------------------------------- ResNet50 Model is trained and saved ------------------------------------------

#### This whole portion can be skipped by importing  *load_model*  and then previously saved model can be used

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [19]:
 !wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-07-10 21:44:44--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip.3’

concrete_data_week4 100%[===================>] 249.37M  29.9MB/s    in 8.3s    

2020-07-10 21:44:53 (30.0 MB/s) - ‘concrete_data_week4.zip.3’ saved [261483817/261483817]



In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [21]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input as pi

In [22]:
num_classes_vgg16 = 2
image_resize_week4 = 224
batch2_size_training = 100
batch2_size_validation = 100

In [23]:
data_generator_2 = ImageDataGenerator(
    preprocessing_function=pi,
)

In [24]:
train_generator_2 = data_generator_2.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize_week4, image_resize_week4),
    batch_size=batch2_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [25]:
validation_generator_2 = data_generator_2.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize_week4, image_resize_week4),
    batch_size=batch2_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [26]:
model_2 = Sequential()
#model = tensorflow.keras.Sequential()

In [27]:
model_2.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [28]:
model_2.add(Dense(num_classes_vgg16, activation='softmax'))

In [29]:
model_2.layers

In [30]:
model_2.layers[0].layers

In [31]:
model_2.layers[0].trainable = False

In [32]:
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [33]:
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

<a id="item43"></a>

In [34]:
steps_per_epoch_training_2 = len(train_generator_2)/ batch2_size_training
steps_per_epoch_validation_2 = len(validation_generator_2)/ batch2_size_validation
num_epochs_2 = 10

In [35]:
fit_history = model_2.fit_generator(
    train_generator_2,
    steps_per_epoch=steps_per_epoch_training_2,
    epochs=num_epochs_2,
    validation_data=validation_generator_2,
    validation_steps=steps_per_epoch_validation_2,
    verbose=1,
)

Epoch 1/10
4/3 [=======================================] - 11s 3s/step - loss: 0.6976 - accuracy: 0.5825 - val_loss: 0.5328 - val_accuracy: 0.6700
Epoch 2/10
4/3 [=======================================] - 3s 745ms/step - loss: 0.5240 - accuracy: 0.6900 - val_loss: 0.4068 - val_accuracy: 0.8000
Epoch 3/10
4/3 [=======================================] - 3s 742ms/step - loss: 0.3373 - accuracy: 0.8675 - val_loss: 0.3038 - val_accuracy: 0.9000
Epoch 4/10
4/3 [=======================================] - 3s 741ms/step - loss: 0.3201 - accuracy: 0.8825 - val_loss: 0.2008 - val_accuracy: 0.9200
Epoch 5/10
4/3 [=======================================] - 3s 732ms/step - loss: 0.2446 - accuracy: 0.9450 - val_loss: 0.1869 - val_accuracy: 0.9300
Epoch 6/10
4/3 [=======================================] - 3s 724ms/step - loss: 0.2075 - accuracy: 0.9500 - val_loss: 0.1690 - val_accuracy: 0.9600
Epoch 7/10
4/3 [=======================================] - 3s 714ms/step - loss: 0.1648 - accuracy: 0.9725 -

In [36]:
model_2.save('classifier_vgg16_model.h5')

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [37]:
test_generator = data_generator_2.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize_week4, image_resize_week4),
    shuffle=False)

Found 500 images belonging to 2 classes.


In [38]:
steps_per_epoch_test_2 = len(test_generator)/ batch2_size_validation

In [39]:
from keras.models import load_model

In [40]:
pretrained_model_1 = load_model('classifier_resnet_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [41]:
pretrained_model_2 = load_model('classifier_vgg16_model.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [42]:
score_1 = pretrained_model_1.evaluate_generator(test_generator, steps=steps_per_epoch_test_2)

In [43]:
score_2 = pretrained_model_2.evaluate_generator(test_generator, steps=steps_per_epoch_test_2)

In [44]:
print('Accuracy of ResNet50: {}% \n Error: {}'.format(score_1[1], 1 - score_1[1]))        
print('Accuracy of VGG16: {}% \n Error: {}'.format(score_2[1], 1 - score_2[1]))        

Accuracy of ResNet50: 0.96875% 
 Error: 0.03125
Accuracy of VGG16: 1.0% 
 Error: 0.0


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [45]:
import numpy as np

In [46]:
predict1 = pretrained_model_1.predict_generator(test_generator, steps=steps_per_epoch_test_2)
predict2 = pretrained_model_2.predict_generator(test_generator, steps=steps_per_epoch_test_2)

In [47]:
# get the dictionary of classes
label2index = test_generator.class_indices

# obtain the list of classes
list_of_label = list(label2index.keys())
print("The list of classes: ", list_of_label)

The list of classes:  ['negative', 'positive']


In [48]:
print("ResNet50 Predictions :")
for i in range(0,5):
    pred_class = np.argmax(predict1[i])
    pred_label = list_of_label[pred_class]
    print('Prediction :{}'.format(
        pred_label,
        predict1[i][pred_class]))

ResNet50 Predictions :
Prediction :negative
Prediction :negative
Prediction :negative
Prediction :negative
Prediction :negative


In [49]:
print("VGG16 Predictions :")
for i in range(0,5):
    pred_class = np.argmax(predict2[i])
    pred_label = list_of_label[pred_class]
    print('Prediction :{}'.format(
        pred_label,
        predict2[i][pred_class]))

VGG16 Predictions :
Prediction :negative
Prediction :negative
Prediction :negative
Prediction :negative
Prediction :negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).